# **Control SDP using the Tango devices**

Controlling Science Data Processor (SDP) using Tango devices allows to directly interact with the devices such as querying,
changing attributes and executing commands directly to Tango.

The Tango devices in the SDP are:
   - The SDP controller Tango device is designed to provide the overall control of the SDP. The commands it receives cause
     the other SDP services to be stopped or started, and its attributes report on the overall state of the system.
   - The SDP subarray Tango device is the principal means by which processing is initiated in the SDP.

If you get stuck you can look at the official documentation pages:
 - [SDP Integration](https://developer.skao.int/projects/ska-sdp-integration/en/latest/index.html)
 - [SDP Local Monitoring and Control](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/index.html)
 - [SDP Processing Scripts](https://developer.skao.int/projects/ska-sdp-script/en/latest/)
 - [SDP on the Developer Portal](https://developer.skao.int/en/latest/projects/area/sdp.html)

If you still don't find the answer to your questions, contact us on Slack: #help-sdp


## 1. Import packages and set environments

SDP contains Tango device servers which control different aspects of the system. The SDP Controller device
provides the top-level control of the system. The SDP Subarray device controls the processing associated
with a telescope subarray.

Each Tango Controls system/deployment has to have at least one running `databaseds` device server. The machine on which
`databaseds` device server is running has a role called Tango Host. `Databaseds` is a device server providing
configuration information to all other components of the system as well as a runtime catalog of the components/devices.
It allows client applications to find devices in distributed environment. `TANGO_HOST` environment variable would need
to be set to point to a remote host that is running the Configuration database.

Import all the required packages and define `namespace`, `databaseds` service and set the `TANGO_HOST`.

In [ ]:
import os
from tango import DeviceProxy, EventType

# specify here the namespace to connect in this cluster
KUBE_NAMESPACE = "<update-with-ns!!!>"

# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the TANGO_HOST
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"


## 2. Accessing the Tango Interface

Let's start by obtaining a handle of the subarray tango device:

In [ ]:
d = DeviceProxy('test-sdp/subarray/02')

Let's check the state of the device

In [ ]:
d.state()

The device should be in `OFF` state. This means the device is in inactive state

Let's check subarray observing state

In [ ]:
d.obsState

The above output would be `EMPTY`. This means no receive and real-time processing resources are assigned to the subarray

Going to set the device into its operational state

In [ ]:
d.On()
d.state()

The state should now be in `ON`. This means it has now been transitioned to operational state.

Now, we need to start the execution block using the `AssignResources` command. It takes an argument which
contains configuration data in JSON format. The data are described by a schema which is versioned to support
evolution of the interfaces. The schema is specified in the argument with the interface keyword. More details about the
schemas can be found [here](https://developer.skao.int/projects/ska-telmodel/en/latest/).

The configuration string defines externally managed resources and an execution block (EB). The EB contains information
about the processing blocks (PBs) and that is required for SDP to receive visibility data from the
correlator beam-former (CBF), provide calibration solutions, receive candidate and timing data from the pulsar search
and timing subsystems and defines scan types.

Below shows the configuration string for the execution block. This configuration string runs one realtime (test_receive_addresses)
and one batch processing (test_dask) processing scripts.

The `test_receive_addresses` script is designed to test the mechanism for generating SDP receive addresses from the channel
link map for each scan type which is contained in the list of scan types in the EB.
This address map get published to the appropriate attribute once the SDP subarray finishes the transition following AssignResources.

The `test-dask` script is designed to test deploying two instances of a Dask execution engine and executing a simple function on each one.

More details about other processing scripts can be found
[here](https://developer.skao.int/projects/ska-sdp-script/en/latest/).


In [ ]:
import json
import random
from datetime import date

generator = "notebook"
today = date.today().strftime("%Y%m%d")
number = random.randint(0, 99998)

EXECUTION_BLOCK_ID = f"eb-{generator}-{today}-{number:05d}"
PROCESSING_BLOCK_ID_REALTIME = f"pb-{generator}-{today}-{number:05d}"
PROCESSING_BLOCK_ID_BATCH = f"pb-{generator}-{today}-{number+1:05d}"


config = {
  "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
  "resources": {
    "receptors": ["SKA001", "SKA002", "SKA003", "SKA004"]
  },
  "execution_block": {
    "eb_id":f"{EXECUTION_BLOCK_ID}",
    "context": {},
    "max_length": 3600.0,
    "beams": [
      {"beam_id": "vis0", "function": "visibilities"}
    ],
    "scan_types": [
      {
        "scan_type_id": ".default",
        "beams": {
          "vis0": {
            "channels_id": "vis_channels",
            "polarisations_id": "all"
          }
        }
      },
      {
        "scan_type_id": "target:a",
        "derive_from": ".default",
        "beams": {
          "vis0": {
            "field_id": "field_a"
          }
        }
      }
    ],
    "channels": [
      {
        "channels_id": "vis_channels",
        "spectral_windows": [
          {
            "spectral_window_id":"fsp_1_channels",
            "count": 4,
            "start": 0,
            "stride": 2,
            "freq_min": 350000000.0,
            "freq_max": 368000000.0,
            "link_map": [[0, 0], [200, 1], [744, 2], [944, 3]]
          }
        ]
      }
    ],
    "polarisations": [
      {
        "polarisations_id": "all",
        "corr_type": ["XX", "XY", "YX", "YY"]
      }
    ],
    "fields": [
      {
        "field_id": "field_a",
        "phase_dir": {
          "ra": [123.0],
          "dec": [-60.0],
          "reference_time": "...",
          "reference_frame": "ICRF3"
        },
        "pointing_fqdn": "..."
      }
    ]
  },
  "processing_blocks": [
    {
      "pb_id": f"{PROCESSING_BLOCK_ID_REALTIME}",
      "script": {"kind": "realtime", "name": "test-receive-addresses", "version": "0.5.0"},
      "sbi_ids": ["sbi-test-20220916-00000"],
      "parameters": {}
    },
    {
      "pb_id": f"{PROCESSING_BLOCK_ID_BATCH}",
      "script": {"kind": "batch", "name": "test-dask", "version": "0.3.0"},
      "parameters": {},
      "dependencies": [
        {"pb_id": f"{PROCESSING_BLOCK_ID_REALTIME}", "kind": ["calibration"]}
      ]
    }
  ]
}

config_eb = json.dumps(config)

The `AssignResources` command assigns resources to the subarray, creates the execution block and the processing blocks
and sets receive addresses.

The `obsState` will be in `RESOURCING` when the resources are being assigned and then will be set to `IDLE`
once the receive and real-time processing resources are assigned to the subarray as specified in the EB.

In [ ]:
d.AssignResources(config_eb)

Once the command above is executed, you need to wait until the `obsState` is set to `IDLE`.


In [ ]:
d.obsState

Let's check if the `receiveAddresses` attribute is set with a JSON-formatted string which contains host addresses
and ports which the CSP subsystems use to send the data. It will also contain names of SDP attributes that provide
real-time calibration information to the rest of the system.


In [ ]:
d.receiveAddresses

`resources` attribute can be used to check if externally managed resources have been set.

In [ ]:
d.resources

The `Configure` command configures scan type for the following scans. The argument it takes specifies the scan type.
It can declare new scan types to add the ones already defined for the execution block.

In [ ]:
d.Configure('{"interface": "https://schema.skao.int/ska-sdp-configure/0.4", "scan_type": "target:a"}')
d.obsState

The `obsState` will be transitioning from `CONFIGURING` when the scan type is being configured and will
be set to `READY` when the scan type is configured and the subarray is ready to scan.

The `Scan` command begins a scan of the configured type. The argument it takes specifies the scan ID.

In [ ]:
d.Scan('{"interface": "https://schema.skao.int/ska-sdp-scan/0.4", "scan_id": 1}')
d.obsState

The `obsState` will be set to `SCANNING` and it begins scanning.

To end the scan, run the following command.

In [ ]:
d.EndScan()
d.obsState

The `obsState` will be set to `READY`.

To end the execution block and the real-time processing blocks in the subarray run the following command.
This ends the real-time processing blocks and batch processing starts once when the resources are available.

In [ ]:
d.End()
d.obsState

The `obsState` should be set to `IDLE`.

The batch processing script will run until it's carried out the sequence of actions and will finish automatically.

To release externally managed resources from the subarray run the following command.

In [ ]:
d.ReleaseResources('{ "interface": "https://schema.skao.int/ska-sdp-releaseres/0.4", '
                   '"resources": {"receptors": ["SKA001", "SKA002", "SKA003", "SKA004"]}}')
d.obsState

In this case the `obsState` will be transitioning from `RESOURCING` to `IDLE` or `EMPTY`. It will transition to `IDLE`
when there are some resources still assigned (for example, four receptors assigned and only two released).

To release all the externally managed resources assigned to the subarray, run the `ReleaseAllResources` command.

To set the device to inactive state (or turn the device off), run the following command

In [ ]:
d.Off()
d.state()

The state should be set to `OFF`.

More details about each of the SDP Subarray commands can be found in the [LMC documentation](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/sdp_subarray.html)

More details about each of the SDP Subarray commands can be found here [here](https://developer.skao.int/projects/ska-sdp-lmc/en/latest/sdp_subarray.html)